In [1]:
login_minio = "admin"
password_minio = "admin_password"
host = "http://localhost:9001" #change localhost to minio in container code

In [2]:
import duckdb

In [ ]:
duckdb.sql("INSTALL httpfs;")
duckdb.sql("LOAD httpfs;")

duckdb.sql("""CREATE OR REPLACE SECRET secret (
    TYPE s3,
    PROVIDER config,
    ENDPOINT '127.0.0.1:9001',
    URL_STYLE 'path',
    USE_SSL false,
    KEY_ID 'admin',
    SECRET 'admin_password',
    REGION 'us-east-1');""")

bucket = 'airflow.learn'
file_path = '2026-01-13 08:37:19.193383+00:00'


In [76]:
binary_file = duckdb.sql(f"""
    SELECT *
    FROM read_json('s3://{bucket}/{file_path}');
""").df()

In [83]:
binary_file

,bars
0,"{'LIN': {'c': 443.65, 'h': 443.705, 'l': 443.5..."


In [91]:
duckdb.sql("""
    WITH parsed AS (
        SELECT bars as bars_json
        FROM binary_file
    ),
    tickers AS (
        SELECT json_keys(bars_json) as ticker_list
        FROM parsed
    ),
    exploded AS (
        SELECT 
            unnest(ticker_list) as ticker,
            json_extract_string(
                (SELECT bars_json FROM parsed LIMIT 1), 
                concat('$.', unnest(ticker_list))
            ) as ticker_data
        FROM tickers
    )
    SELECT 
        ticker as ticker,
        json_extract_string(ticker_data, '$.c')::DOUBLE as close,
        json_extract_string(ticker_data, '$.h')::DOUBLE as high,
        json_extract_string(ticker_data, '$.l')::DOUBLE as low,
        json_extract_string(ticker_data, '$.n')::INTEGER as n,
        json_extract_string(ticker_data, '$.o')::DOUBLE as open,
        json_extract_string(ticker_data, '$.t') as time,
        json_extract_string(ticker_data, '$.v')::INTEGER as volume,
        json_extract_string(ticker_data, '$.vw')::DOUBLE as vwap
    FROM exploded
""").df()

,ticker,close,high,low,n,open,time,volume,vwap
0,lin,443.650,443.705,443.550,115,443.620,2026-01-12 20:59:00,3160,443.647292
1,cost,943.350,943.590,943.110,133,943.280,2026-01-12 20:59:00,3793,943.258824
2,lly,1081.015,1081.240,1080.790,102,1080.790,2026-01-12 20:59:00,2708,1080.940634
3,cnc,46.390,46.430,46.390,128,46.430,2026-01-12 20:59:00,5152,46.415255
4,fds,297.125,297.170,297.125,62,297.140,2026-01-12 20:59:00,586,297.140359
...,...,...,...,...,...,...,...,...,...
498,ffiv,270.590,270.700,270.505,24,270.700,2026-01-12 20:59:00,781,270.658400
499,ntrs,146.225,146.225,146.225,23,146.225,2026-01-12 20:59:00,416,146.225000
500,xyl,139.050,139.050,138.900,46,138.900,2026-01-12 20:59:00,1747,138.989286
501,bsx,95.680,95.700,95.640,266,95.685,2026-01-12 20:59:00,12302,95.681151


In [100]:
table_final = duckdb.sql("""
    WITH parsed AS (
        SELECT bars as bars_json
        FROM binary_file
    ),
    tickers AS (
        SELECT json_keys(bars_json) as ticker_list
        FROM parsed
    ),
    exploded AS (
        SELECT 
            unnest(ticker_list) as ticker,
            json_extract_string(
                (SELECT bars_json FROM parsed LIMIT 1), 
                concat('$.', unnest(ticker_list))
            ) as ticker_data
        FROM tickers
    )
    SELECT 
        ticker as ticker,
        json_extract_string(ticker_data, '$.c')::DOUBLE as close,
        json_extract_string(ticker_data, '$.h')::DOUBLE as high,
        json_extract_string(ticker_data, '$.l')::DOUBLE as low,
        json_extract_string(ticker_data, '$.n')::INTEGER as n,
        json_extract_string(ticker_data, '$.o')::DOUBLE as open,
        json_extract_string(ticker_data, '$.t') as time,
        json_extract_string(ticker_data, '$.v')::INTEGER as volume,
        json_extract_string(ticker_data, '$.vw')::DOUBLE as vwap
    FROM exploded
""")


┌─────────┬──────────┬─────────┬─────────┬───────┬─────────┬─────────────────────┬────────┬─────────────┐
│ ticker  │  close   │  high   │   low   │   n   │  open   │        time         │ volume │    vwap     │
│ varchar │  double  │ double  │ double  │ int32 │ double  │       varchar       │ int32  │   double    │
├─────────┼──────────┼─────────┼─────────┼───────┼─────────┼─────────────────────┼────────┼─────────────┤
│ LIN     │   443.65 │ 443.705 │  443.55 │   115 │  443.62 │ 2026-01-12 20:59:00 │   3160 │  443.647292 │
│ COST    │   943.35 │  943.59 │  943.11 │   133 │  943.28 │ 2026-01-12 20:59:00 │   3793 │  943.258824 │
│ LLY     │ 1081.015 │ 1081.24 │ 1080.79 │   102 │ 1080.79 │ 2026-01-12 20:59:00 │   2708 │ 1080.940634 │
│ CNC     │    46.39 │   46.43 │   46.39 │   128 │   46.43 │ 2026-01-12 20:59:00 │   5152 │   46.415255 │
│ FDS     │  297.125 │  297.17 │ 297.125 │    62 │  297.14 │ 2026-01-12 20:59:00 │    586 │  297.140359 │
│ AON     │   350.46 │  350.88 │  350.46 │    

In [106]:
silver_bucket = "stocks.silver"
duckdb.sql(f"""
COPY table_final 
TO 's3://{silver_bucket}/{file_path}' 
(FORMAT parquet)
""")